In [1]:
import os 
import json
import re

In [2]:
exp_name = "MiniCPM"
flag = 'all'

In [3]:
if flag == 'train':
    ran = range(1,91)
elif flag == 'test':
    ran = range(91,101)
else:
    ran = range(1,101)

In [ ]:
def parse_execution_history(history_string):
    history_string = history_string.replace("Historical Execution: ", "")
    historty_splits = re.split(r'(\(fail\)|\(success\))\s', history_string)

    parsed_steps = []

    for i in range(0, len(historty_splits)-1, 2):
        result = 'success' if 'success' in historty_splits[i+1] else 'fail'
        action_parts = re.sub(r'\(.*?\)', '', historty_splits[i], 1).strip().lstrip('[').rstrip(']').split(',')
        
        parsed_step = [
            action_parts[0].strip(),
            action_parts[1].strip().lower() if len(action_parts) > 1 else "None",
            action_parts[2].strip().lower() if len(action_parts) > 2 else "None",
            result
        ]
        parsed_steps.append(parsed_step)

    return parsed_steps


In [ ]:
def eval_success_list_gen(exp_name,round):

    exp_path = os.path.join("exp", exp_name)

    eval_success_list = []

    for index in os.listdir(exp_path):
        conv_path = os.path.join(exp_path, index, f"{round}/conversation.json")
        if not os.path.exists(conv_path):
            continue

        with open(conv_path, 'r') as file:
            conv = json.load(file)
        if 'End' in conv[-1]:   
            exe_dict = {}
            exe_dict["index"] = int(index) 
            exe_dict["step_len"] = len(conv)

            history = conv[-1].split('\n')[3]
            exe_dict["history"] = parse_execution_history(history)
            exe_dict["history"].append(["End", "None", "None", "success"])
            eval_success_list.append(exe_dict) 

    eval_success_list = sorted(eval_success_list, key=lambda x: x["index"])
    #print(len(eval_success_list))

    return eval_success_list

def load_name_mapping():
        with open('hab-mobile-manipulation/name_dict.txt', 'r') as file:
            content = file.read()
        lines = content.split('\n')

        name_dict = {}
        for i in range(0, len(lines), 3):
            value = lines[i].strip().strip(':')
            key = lines[i + 1].strip()
            keys = key.split('/')
            for i in keys:
                name_dict[i] = value

        return name_dict
    # mapping item name
mapping_dict = load_name_mapping()

def match_action(action, keypoint, mapping_dict):
        #print(keypoint)
        if action[3] == 'fail':
               return False

        key_action = [element.strip() for element in keypoint.strip('[').strip(']').split(',')]
        if len(key_action) == 2:
            key_action.append("None")
        if action[0] == key_action[0]:
                if key_action[0] == "End":
                    return True
                obj1 = mapping_dict[action[1]]
                obj2 = mapping_dict[key_action[1]]
                if obj1 == obj2:
                       obj11 = "None" if action[2] == "None" else mapping_dict[action[2]]
                       obj22 = "None" if key_action[2] == "None" else mapping_dict[key_action[2]]
                       if obj11 == obj22:
                              return True
        return False

def match(list1, list2):
    if len(list2) == 0:
        return False
    index1 = 0
    index2 = 0

    while index1 < len(list1) and index2 < len(list2):
        if match_action(list1[index1], list2[index2], mapping_dict):
            index2 += 1
        index1 += 1
    
    return index2 == len(list2)


In [ ]:
def eval_list_gen_ran(exp_name,round,ran):
    exp_path = os.path.join("exp", exp_name)

    eval_list = []

    for index in os.listdir(exp_path):
        conv_path = os.path.join(exp_path, index, f"{round}/conversation.json")
        if not os.path.exists(conv_path):
            continue

        if int(index) in ran:
            with open(conv_path, 'r') as file:
                conv = json.load(file)  
            exe_dict = {}
            exe_dict["index"] = int(index) 
            exe_dict["step_len"] = len(conv)

            history = conv[-1].split('\n')[3]
            exe_dict["history"] = parse_execution_history(history)
            if 'End' in conv[-1]: 
                exe_dict["history"].append(['End'])
            eval_list.append(exe_dict) 

    eval_list = sorted(eval_list, key=lambda x: x["index"])
    #print(len(eval_list))

    return eval_list

def ser(exp_name, ran):
    success_end = 0
    end_count = 0
    for round in range(1, 4):
        eval_list = eval_list_gen_ran(exp_name,round,ran)
        eval_success_list = eval_success_list_gen(exp_name,round)

        success_list = []

        for eval_success in eval_success_list:
            task_id = eval_success['index']
            history = eval_success['history']
            if 1 <= int(task_id) <= 90:
                data_dir = "EMMOE-100/data/train"
            else:
                data_dir = "EMMOE-100/data/test"

            with open(os.path.join(data_dir, str(task_id), "keypath.json"), 'r') as file:
                keypaths = json.load(file)

            for keypath in keypaths:
                if match(history, keypath):
                    success_list.append(task_id)
                    break

        #print(len(success_list))
        for i in eval_list:
            if ['End'] in i['history']:
                end_count += 1
                if i['index'] in success_list:
                    success_end += 1
    print(success_end, end_count)
    
    return success_end/end_count

In [7]:
ser(exp_name,ran)

2 5


0.4